In [2]:
import numpy as np
from pprint import pprint
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

### 1. 텍스트 토큰화
print('='*30, '1. 텍스트 토큰화', '='*30)

# 전처리 텍스트
text = '해보지 않으면 해낼 수 없다'

# 해당 텍스트 토큰화
result = text_to_word_sequence(text)
print('\n원본 문장 :', text)
print('토큰화 :', result)

# 전처리 문장
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화 합니다. ', 
        '텍스트의 단어로 토큰화 해야 딥러닝에서 인식됩니다. ', 
        '토큰화 한 결과는 딥러닝에서 사용할 수 있습니다. ']

# 해당 문장 토큰화
token = Tokenizer() # 토큰화 함수 지정
token.fit_on_texts(docs) # 토큰화 함수에 문장 적용

print('\n원본 문장 :', docs)
print('문장 개수 :', token.document_count)

print('\n==== 출력 순서 랜덤 ====')
print('\n단어 카운트 :')
pprint(token.word_counts)

print('\n각 단어가 몇 개의 문장에 포함되어 있는지 :')
pprint(token.word_docs)

print('\n각 단어에 매겨진 인덱스 값 :')
pprint(token.word_index)



### 2. 단어 원-핫 인코딩
print('\n','='*30, '2. 단어의 원-핫 인코딩', '='*30)

text = "오랫동안 꿈꾸는 이는 그 꿈을 닮아간다"
print('\n원본 문장 :', text)

token = Tokenizer()
token.fit_on_texts([text])
print('\n각 단어의 인덱스 값 :', token.word_index)

# texts_to_sequences() 함수 : 각 단어를 원-핫 인코딩 방식으로 변환
x = token.texts_to_sequences([text])
print('\n각 단어 원핫 인코딩 방식 적용 :', x)

from tensorflow.keras.utils import to_categorical

# 인덱스 수에 하나를 추가해서 원-핫 인코딩 배열 생성
# 배열 맨 앞에 0이 추가되므로 단어 수보다 1이 더 많에 인덱스 숫자 설정 필요
word_size = len(x[0]) + 1
x = to_categorical(x, num_classes=word_size)
print('\n원핫 인코딩 배열 생성 :\n', x)



### 3. 단어 임베딩

print('\n','='*30, '3. 단어 임베딩', '='*30)
print('\n단어 임베딩은 주어진 배열을 정해진 길이로 압축 시키는 것')
print('Embedding(입력될 단어의 개수, 임베딩 후 출력되는 벡터 크기, 단어를 매번 얼마나 입력할지)')



### 4. 텍스트 읽고 긍부정 예측

print('\n','='*30, '4. 텍스트 읽고 긍부정 예측', '='*30)

# 텍스트 리뷰 자료 지정
docs = ["너무 재밌네요", "최고예요", "참 잘 만든 영화예요", "추천하고 싶은 영화입니다", "한번 더 보고싶네요",
        "글쎄요", "별로예요", "생각보다 지루하네요", "연기가 어색해요", "재미없어요"]

# 긍정 리뷰는 1, 부정 리뷰는 0으로 클래스 지정
classes = np.array([1,1,1,1,1,0,0,0,0,0])

# 해당 텍스트 토큰화
token = Tokenizer()
token.fit_on_texts(docs)
print('\n원본 문장 :', docs)
print('\n각 단어의 인덱스 값 :')
pprint(token.word_index)

# 토큰에 지정된 인덱스로 새로운 배열 생성
x = token.texts_to_sequences(docs)
print('\n인덱스 배열 새로 생성 :', x)

# 패딩(padding) : pad_sequence()함수 사용
# 데이터 길이 동일하게 설정, 설정 길이보다 짧은 데이터은 부족한 길이 만큼 숫자 0으로 대체, 긴 데이터 경우 설정 길이를 제외한 데이터 삭제
padded_x = pad_sequences(x, 4)
print('\npadding 결과 :\n', padded_x)


print('\n딥러닝 모델 시작')

# 임베딩 함수(입력, 출력, 단어 수)
        # 입력 : 총 몇개의 단어집합에서 입력할 것인지
        # 출력 : 몇 개의 임베딩 결과 사용할 것인지
        # 단어 수 : 매번 입력된 단어수는 몇개로 할 것인지
# 배열 맨 앞에 0이 추가되므로 단어 수보다 1이 더 많에 인덱스 숫자 설정 필요
word_size = len(token.word_index) + 1

model = Sequential()
model.add(Embedding(word_size, 8, input_length=4))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(padded_x, classes, epochs=20)

print("\n Accuracy: %.4f" % (model.evaluate(padded_x, classes)[1]))


============================== 1. 텍스트 토큰화 ==============================

원본 문장 : 해보지 않으면 해낼 수 없다
토큰화 : ['해보지', '않으면', '해낼', '수', '없다']

원본 문장 : ['먼저 텍스트의 각 단어를 나누어 토큰화 합니다. ', '텍스트의 단어로 토큰화 해야 딥러닝에서 인식됩니다. ', '토큰화 한 결과는 딥러닝에서 사용할 수 있습니다. ']
문장 개수 : 3

==== 출력 순서 랜덤 ====

단어 카운트 :
OrderedDict([('먼저', 1),
             ('텍스트의', 2),
             ('각', 1),
             ('단어를', 1),
             ('나누어', 1),
             ('토큰화', 3),
             ('합니다', 1),
             ('단어로', 1),
             ('해야', 1),
             ('딥러닝에서', 2),
             ('인식됩니다', 1),
             ('한', 1),
             ('결과는', 1),
             ('사용할', 1),
             ('수', 1),
             ('있습니다', 1)])

각 단어가 몇 개의 문장에 포함되어 있는지 :
defaultdict(<class 'int'>,
            {'각': 1,
             '결과는': 1,
             '나누어': 1,
             '단어로': 1,
             '단어를': 1,
             '딥러닝에서': 2,
             '먼저': 1,
             '사용할': 1,
             '수': 1,
             '인식됩니다': 1,
             '있습니다': 1,
            